# 2017 US Individual Market Medical Insurance

Data from [healthcare.gov](https://data.healthcare.gov/dataset/2017-QHP-Landscape-SHOP-Market-Medical/sm8s-t6mi). 

# Upload Data

In [1]:
# upload zip
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving QHP_Landscape_Individual_Market_Medical.zip to QHP_Landscape_Individual_Market_Medical.zip
User uploaded file "QHP_Landscape_Individual_Market_Medical.zip" with length 12959999 bytes


In [2]:
# upload state population data
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving US_Population_Density_by_State.csv to US_Population_Density_by_State.csv
User uploaded file "US_Population_Density_by_State.csv" with length 2161 bytes


In [31]:
# upload county FIPS data
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving FIPS_County_Codes_Alt.csv to FIPS_County_Codes_Alt.csv
User uploaded file "FIPS_County_Codes_Alt.csv" with length 61679 bytes


In [4]:
!ls

FIPS_County_Codes.csv			     sample_data
QHP_Landscape_Individual_Market_Medical.zip  US_Population_Density_by_State.csv


In [5]:
# unzip data to directory "data"
!unzip QHP_Landscape_Individual_Market_Medical.zip

Archive:  QHP_Landscape_Individual_Market_Medical.zip
  inflating: QHP_Landscape_Individual_Market_Medical.csv  


In [0]:
!rm QHP_Landscape_Individual_Market_Medical.zip

In [0]:
# !rm FIPS_County_Codes_Alt.csv

In [32]:
!ls

FIPS_County_Codes_Alt.csv		     sample_data
QHP_Landscape_Individual_Market_Medical.csv  US_Population_Density_by_State.csv


# Imports

In [33]:
# standard libraries
import pandas as pd
import numpy as np
import os

# visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
from vega_datasets import data

# train-test split
from sklearn.model_selection import train_test_split

# linear regression models
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, LinearSVR
from xgboost import XGBRegressor

# cross val, k-folds, scaler
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

# suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# ignore warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
warnings.filterwarnings("ignore", category=Warning)
print('Warnings will be ignored!')

Warnings will be ignored!


# Population Stats

In [0]:
# population stats df
pop_df = pd.read_csv('US_Population_Density_by_State.csv')
pop_df.columns = ['State Name', 'State', 'FIPS State Code', 'Pop Density', 'Population', 'Area']

pop_df['Pop Density'] = pop_df['Pop Density'].str.replace(',', '')
pop_df['Pop Density'] = pop_df['Pop Density'].astype('uint32')

pop_df['Population'] = pop_df['Population'].str.replace(',', '')
pop_df['Population'] = pop_df['Population'].astype('uint32')

pop_df['Area'] = pop_df['Area'].str.replace(',', '')
pop_df['Area'] = pop_df['Area'].astype('uint32')

In [0]:
# dictionary for state names and codes for visualizations
state_geo_dict = pd.Series(pop_df.State.values,index=pop_df['FIPS State Code']).to_dict()

# County FIPS Data

In [0]:
fips_df = pd.read_csv('FIPS_County_Codes_Alt.csv')

# county changed to county + state code
fips_df['County'] = fips_df['Name'] + fips_df['State']
fips_df["County"] = fips_df['County'].str.upper()

In [0]:
county_fips_dict = pd.Series(fips_df.FIPS.values,index=fips_df.County).to_dict()

In [38]:
fips_df.head(10)

,FIPS,Name,State,County
0,1001,Autauga,AL,AUTAUGA AL
1,1003,Baldwin,AL,BALDWIN AL
2,1005,Barbour,AL,BARBOUR AL
3,1007,Bibb,AL,BIBB AL
4,1009,Blount,AL,BLOUNT AL
5,1011,Bullock,AL,BULLOCK AL
6,1013,Butler,AL,BUTLER AL
7,1015,Calhoun,AL,CALHOUN AL
8,1017,Chambers,AL,CHAMBERS AL
9,1019,Cherokee,AL,CHEROKEE AL


# Data Cleaning + Feature Engineering

In [0]:
# load data
df = pd.read_csv('QHP_Landscape_Individual_Market_Medical.csv')

# remove child only offerings
df = df[df['Child Only Offering'] == 'Allows Adult and Child-Only']

df = df.rename(index=str, columns={'Premium Couple 21  ': "Premium Couple 21"})

# drop features with all nulls and unneeded features
df = df.drop(['Premium Scenarios', 'Standard Plan Cost Sharing', 'Child Only Offering', 
             'Customer Service Phone Number TTY', 'Customer Service Phone Number Toll Free', 
             'Customer Service Phone Number Local', 'Adult Dental', '73 Percent Actuarial Value Silver Plan Cost Sharing', 
             '87 Percent Actuarial Value Silver Plan Cost Sharing', '94 Percent Actuarial Value Silver Plan Cost Sharing', 
             'Medical Maximum Out Of Pocket - individual - 73 percent', 'Drug Maximum Out of Pocket - individual - 73 percent', 
             'Medical Maximum Out of Pocket - family - 73 percent', 'Drug Maximum Out of Pocket - Family - 73 percent', 
             'Primary Care Physician - 73 percent', 'Specialist - 73 percent', 'Emergency Room - 73 percent', 
             'Inpatient Facility - 73 percent', 'Inpatient Physician - 73 percent', 'Generic Drugs - 73 percent', 
             'Preferred Brand Drugs - 73 percent', 'Non-preferred Brand Drugs - 73 percent', 'Specialty Drugs - 73 percent', 
             'Medical Deductible - individual - 87 percent', 'Drug Deductible - individual - 87 percent', 
             'Medical Deductible - family - 87 percent', 'Drug Deductible - family - 87 percent', 
             'Medical Maximum Out Of Pocket - individual - 87 percent', 'Drug Maximum Out of Pocket - individual - 87 percent', 
             'Medical Maximum Out of Pocket - family - 87 percent', 'Drug Maximum Out of Pocket - Family - 87 percent', 
             'Primary Care Physician - 87 percent', 'Specialist - 87 percent', 'Emergency Room - 87 percent', 
             'Inpatient Facility - 87 percent', 'Inpatient Physician - 87 percent', 'Generic Drugs - 87 percent', 
             'Preferred Brand Drugs - 87 percent', 'Non-preferred Brand Drugs - 87 percent', 'Specialty Drugs - 87 percent', 
             'Medical Deductible - individual - 94 percent', 'Drug Deductible - individual - 94 percent', 
             'Medical Deductible - family - 94 percent', 'Drug Deductible - family - 94 percent', 
             'Medical Maximum Out Of Pocket -individual - 94 percent', 'Drug Maximum Out of Pocket - individual - 94 percent', 
             'Medical Maximum Out of Pocket - family - 94 percent', 'Drug Maximum Out of Pocket - Family  - 94 percent', 
             'Primary Care Physician - 94 percent', 'Specialist - 94 percent', 'Emergency Room - 94 percent', 
             'Inpatient Facility  - 94 percent', 'Inpatient Physician  - 94 percent', 'Generic Drugs - 94 percent', 
             'Preferred Brand Drugs - 94 percent', 'Non-preferred Brand Drugs - 94 percent', 'Specialty Drugs - 94 percent',
             'Drug Formulary URL', 'Summary of Benefits URL', 'Plan Brochure URL'], axis=1)

# reset index
df = df.reset_index(drop=True)

# merge with population data
df = pd.merge(df, pop_df, on='State')

# county changed to county + state code
df['County'] = df['County'] + " " + df['State']
df['County'] = df['County'].str.upper()
df['County'] = df['County'].str.replace('MIAMI-DADE FL', 'DADE FL')
df['County'] = df['County'].str.replace('BRISTOL BAY BOROUGH AK', 'BRISTOL BAY AK')
df['County'] = df['County'].str.replace('.', '')
df['County'] = df['County'].str.replace('MC ', 'MC')
df['County'] = df['County'].str.replace('NORTHUMBERLND', 'NORTHUMBERLAND')
df['County'] = df['County'].str.replace('E BATON ROUGE LA', 'EAST BATON ROUGE LA')
df['County'] = df['County'].str.replace('W BATON ROUGE LA', 'WEST BATON ROUGE LA')
df['County'] = df['County'].str.replace('ST JOHN BAPTIST LA', 'ST JOHN THE BAPTIST LA')
df['County'] = df['County'].str.replace('OBRIEN IA', 'O BRIEN IA')
df['County'] = df['County'].str.replace('MANASSUS', 'MANASSAS')
df['County'] = df['County'].str.replace('LAPAZ AZ', 'LA PAZ AZ')
df['County'] = df['County'].str.replace('MENOMONEE WI', 'MENOMINEE WI')
df['County'] = df['County'].str.replace('POQUOSON VA', 'POQUOSON CITY VA')
df['County'] = df['County'].str.replace('LAGRANGE IN', 'LA GRANGE IN')
df['County'] = df['County'].str.replace('SCOTT BLUFF NE', 'SCOTTS BLUFF NE')
df['County'] = df['County'].str.replace('NORTHWEST ARTIC AK', 'NORTHWEST ARCTIC AK')

# add county FIPS codes
df['FIPS County Code'] = df['County'].map(county_fips_dict)

# split url
df['Network URL'] = df['Network URL'].str.split('/').str[2]

# state value counts
df['StateFreq'] = df['State'].map(df['State'].value_counts())

# turn 'Rating Area' into numeric value
df['Rating Area'] = df['Rating Area'].str.replace('Rating Area ', '').astype('uint16')


# replace nulls in coverage categories
df['Child Dental'] = df['Child Dental'].fillna(0)

# turn coverage categories into binomials
df['Child Dental'] = df['Child Dental'].replace('X', 1)

# convert coverage categories into 'uint8' data type
df['Child Dental'] = df['Child Dental'].astype('uint8')

# convert deductibles into ints / impute 'Not Applicable' values with .quantile(q=0.75) 
df['Medical Deductible - individual - standard'] = df['Medical Deductible - individual - standard'].str.replace('$', '')
df['Medical Deductible - individual - standard'] = df['Medical Deductible - individual - standard'].str.replace(' ', '')
df['Medical Deductible - individual - standard'] = pd.to_numeric(df['Medical Deductible - individual - standard'], errors='coerce')
df['Medical Deductible - individual - standard'] = df['Medical Deductible - individual - standard'].fillna(df['Medical Deductible - individual - standard'].quantile(q=0.75))
df['Medical Deductible - individual - standard'] = df['Medical Deductible - individual - standard'].astype('uint16')

df['Drug Deductible - individual - standard'] = df['Drug Deductible - individual - standard'].str.replace('$', '')
df['Drug Deductible - individual - standard'] = df['Drug Deductible - individual - standard'].str.replace(' ', '')
df['Drug Deductible - individual - standard'] = pd.to_numeric(df['Drug Deductible - individual - standard'], errors='coerce')
df['Drug Deductible - individual - standard'] = df['Drug Deductible - individual - standard'].fillna(df['Drug Deductible - individual - standard'].quantile(q=0.75))
df['Drug Deductible - individual - standard'] = df['Drug Deductible - individual - standard'].astype('uint16')

def convert_deductibles(feature):
    df[feature] = df[feature].str.replace('$', '')
    df[feature] = df[feature].str.replace(',', '')
    df[feature] = df[feature].str.replace(' ', '')
    df[feature] = df[feature].str.replace('Included in Medical', '0')    
    df[feature] = pd.to_numeric(df[feature], errors='coerce')
    df[feature] = df[feature].fillna(df[feature].quantile(q=0.75))
    df[feature] = df[feature].astype('uint16')
    
    return df[feature]
    
df['Medical Deductible -family - standard'] = convert_deductibles('Medical Deductible -family - standard')
df['Drug Deductible - family - standard'] = convert_deductibles('Drug Deductible - family - standard')
df['Medical Maximum Out Of Pocket - individual - standard'] = convert_deductibles('Medical Maximum Out Of Pocket - individual - standard')
df['Drug Maximum Out of Pocket - individual - standard'] = convert_deductibles('Drug Maximum Out of Pocket - individual - standard')
df['Medical Maximum Out of Pocket - family - standard'] = convert_deductibles('Medical Maximum Out of Pocket - family - standard')
df['Drug Maximum Out of Pocket - Family  - standard'] = convert_deductibles('Drug Maximum Out of Pocket - Family  - standard')
df['Medical Deductible - individual - 73 percent'] = convert_deductibles('Medical Deductible - individual - 73 percent')
df['Drug Deductible - individual - 73 percent'] = convert_deductibles('Drug Deductible - individual - 73 percent')
df['Medical Deductible - family - 73 percent'] = convert_deductibles('Medical Deductible - family - 73 percent')
df['Drug Deductible - family - 73 percent'] = convert_deductibles('Drug Deductible - family - 73 percent')


# create average premium per person metric as sum of all premiums divided by estimated # of individuals
df['Avg Premium'] = df[['Premium Child','Premium Adult Individual Age 21', 'Premium Adult Individual Age 27',
       'Premium Adult Individual Age 30', 'Premium Adult Individual Age 40',
       'Premium Adult Individual Age 50', 'Premium Adult Individual Age 60',
       'Premium Couple 21', 'Premium Couple 30', 'Premium Couple 40',
       'Premium Couple 50', 'Premium Couple 60', 'Couple+1 child, Age 21',
       'Couple+1 child, Age 30', 'Couple+1 child, Age 40',
       'Couple+1 child, Age 50', 'Couple+2 children, Age 21',
       'Couple+2 children, Age 30', 'Couple+2 children, Age 40',
       'Couple+2 children, Age 50', 'Couple+3 or more Children, Age 21',
       'Couple+3 or more Children, Age 30',
       'Couple+3 or more Children, Age 40',
       'Couple+3 or more Children, Age 50', 'Individual+1 child, Age 21',
       'Individual+1 child, Age 30', 'Individual+1 child, Age 40',
       'Individual+1 child, Age 50', 'Individual+2 children, Age 21',
       'Individual+2 children, Age 30', 'Individual+2 children, Age 40',
       'Individual+2 children, Age 50',
       'Individual+3 or more children, Age 21',
       'Individual+3 or more children, Age 30',
       'Individual+3 or more children, Age 40',
       'Individual+3 or more children, Age 50',]].sum(axis = 1, skipna = True) / 101


# encode state names by frequency
state_list = df['State'].value_counts().index.tolist()

state_dict = dict()
for x in range(0, (len(state_list))):
    state_dict[state_list[x]] = x
      
df['StateID'] = df['State'].map(state_dict).astype('uint8')

# encode county names by frequency
county_list = df['County'].value_counts().index.tolist()

county_dict = dict()
for x in range(0, (len(county_list))):
    county_dict[county_list[x]] = x
    
df['CountyID'] = df['County'].map(county_dict).astype('uint16')

# encode issuers by frequency
issuer_list = df['Issuer Name'].value_counts().index.tolist()

issuer_dict = dict()
for x in range(0, (len(issuer_list))):
    issuer_dict[issuer_list[x]] = x
    
df['IssuerID'] = df['Issuer Name'].map(issuer_dict).astype('uint16')

# encode marketing plans
marketing_plan_list = df['Plan Marketing Name']

marekting_plan_dict = dict()
for x in range(0, (len(marketing_plan_list))):
    marekting_plan_dict[marketing_plan_list[x]] = x
    
df['PlanMarketingID'] = df['Plan Marketing Name'].map(marekting_plan_dict).astype('uint16')

# network URL encoded
network_url_list = df['Network URL'].value_counts().index.tolist()

network_url_dict = dict()
for x in range(0, (len(network_url_list))):
    network_url_dict[network_url_list[x]] = x
    
df['Network URL ID'] = df['Network URL'].map(network_url_dict).astype('uint16')

# turn 'Source' into binomial
df['Source Encoded'] = df['Source']
df['Source Encoded'] = df['Source Encoded'].replace('HIOS', 1)
df['Source Encoded'] = df['Source Encoded'].replace('SERFF', 2)
df['Source Encoded'] = df['Source Encoded'].replace('OPM', 0)
df['Source Encoded'] = df['Source Encoded'].astype('uint8')

# 'Metal Level' encoding by quality of plan
df['Metal Level Encoded'] = df['Metal Level']
df['Metal Level Encoded'] = df['Metal Level Encoded'].replace('Platinum', 4)
df['Metal Level Encoded'] = df['Metal Level Encoded'].replace('Gold', 3)
df['Metal Level Encoded'] = df['Metal Level Encoded'].replace('Silver', 2)
df['Metal Level Encoded'] = df['Metal Level Encoded'].replace('Bronze', 1)
df['Metal Level Encoded'] = df['Metal Level Encoded'].replace('Catastrophic', 0)
df['Metal Level Encoded'] = df['Metal Level Encoded'].astype('uint8')

# encode plan type
df['Plan Type Encoded'] = df['Plan Type']
df['Plan Type Encoded'] = df['Plan Type Encoded'].replace('PPO', 2)
df['Plan Type Encoded'] = df['Plan Type Encoded'].replace('EPO', 1)
df['Plan Type Encoded'] = df['Plan Type Encoded'].replace('POS', 1)
df['Plan Type Encoded'] = df['Plan Type Encoded'].replace('HMO', 0)

# insurance plans per capita; measure of competition
df['PlansPer100K'] = (df['StateFreq'] / df['Population']) * 100000

# pop density bins
df['PopDensityBins'] = pd.qcut(df['Pop Density'], q=10, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], duplicates='drop')



def mean_premium_map(feature):
    mean_premium = df.groupby(feature, as_index=True)['Avg Premium'].mean()
    mean_premium_dict = dict(zip(mean_premium.index, mean_premium))
    return mean_premium_dict

# mean premium per provider
df['Issuer_MP'] = df['IssuerID'].map(mean_premium_map(df['IssuerID'])).astype('float32')

# mean premium by state
df['State_MP'] = df['StateID'].map(mean_premium_map(df['StateID'])).astype('float32')

# mean premium by county
df['County_MP'] = df['CountyID'].map(mean_premium_map(df['CountyID'])).astype('float32')

# mean premium by rating area
df['RatingArea_MP'] = df['Rating Area'].map(mean_premium_map(df['Rating Area'])).astype('float32')

# mean premium by plans per capita
df['PlansPer100K_MP'] = df['PlansPer100K'].map(mean_premium_map(df['PlansPer100K'])).astype('float32')

# mean premium by marketing id
df['MarketingID_MP'] = df['PlanMarketingID'].map(mean_premium_map(df['PlanMarketingID'])).astype('float32')

# mean premium by pop density
df['PopDensity_MP'] = df['Pop Density'].map(mean_premium_map(df['Pop Density'])).astype('float32')


# mean premium by pop density bucket
df['PopDensityBin_MP'] = df['PopDensityBins'].map(mean_premium_map(df['PopDensityBins'])).astype('float32')


# drop some redundant columns
df = df.drop(['Source'], axis=1)

# drop some difficult to transform features for now; might revisit in the future
df = df.drop(['Primary Care Physician  - standard', 'Specialist  - standard',
       'Emergency Room  - standard', 'Inpatient Facility  - standard',
       'Inpatient Physician - standard', 'Generic Drugs - standard',
       'Preferred Brand Drugs - standard',
       'Non-preferred Brand Drugs - standard', 'Specialty Drugs - standard'], axis=1)




In [40]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75337 entries, 0 to 75336
Data columns (total 84 columns):
State                                                    75337 non-null object
County                                                   75337 non-null object
Metal Level                                              75337 non-null object
Issuer Name                                              75337 non-null object
Plan ID - Standard Component                             75337 non-null object
Plan Marketing Name                                      75337 non-null object
Plan Type                                                75337 non-null object
Rating Area                                              75337 non-null uint16
Network URL                                              75337 non-null object
Child Dental                                             75337 non-null uint8
Premium Child                                            75337 non-null float64
Premium Adult Individu

In [0]:
# # print value counts for all 'objects'
# def object_vcs_and_nulls(df):
#   for i in df:
#     if df[i].dtype == 'O':
# #       if df[i].isnull().sum() > 0:
#         print(df[i].value_counts())  
#         print("Number of Null Values: " + str(df[i].isnull().sum()))
#         print("Percentage of Nulls = " + str(np.round((df[i].isnull().sum() / df.shape[0] * 100), 2)) + "%")
#         print("\n")
      
# object_vcs_and_nulls(df)

In [200]:
print(df.head().to_string())

  State            County Metal Level                               Issuer Name Plan ID - Standard Component                                Plan Marketing Name Plan Type  Rating Area      Network URL  Child Dental  Premium Child  Premium Adult Individual Age 21  Premium Adult Individual Age 27  Premium Adult Individual Age 30  Premium Adult Individual Age 40  Premium Adult Individual Age 50  Premium Adult Individual Age 60  Premium Couple 21  Premium Couple 30  Premium Couple 40  Premium Couple 50  Premium Couple 60  Couple+1 child, Age 21  Couple+1 child, Age 30  Couple+1 child, Age 40  Couple+1 child, Age 50  Couple+2 children, Age 21  Couple+2 children, Age 30  Couple+2 children, Age 40  Couple+2 children, Age 50  Couple+3 or more Children, Age 21  Couple+3 or more Children, Age 30  Couple+3 or more Children, Age 40  Couple+3 or more Children, Age 50  Individual+1 child, Age 21  Individual+1 child, Age 30  Individual+1 child, Age 40  Individual+1 child, Age 50  Individual+2 children

In [28]:
county_fips_dict

{'AUTAUGA AL': 1001,
 'BALDWIN AL': 1003,
 'BARBOUR AL': 1005,
 'BIBB AL': 1007,
 'BLOUNT AL': 1009,
 'BULLOCK AL': 1011,
 'BUTLER AL': 1013,
 'CALHOUN AL': 1015,
 'CHAMBERS AL': 1017,
 'CHEROKEE AL': 1019,
 'CHILTON AL': 1021,
 'CHOCTAW AL': 1023,
 'CLARKE AL': 1025,
 'CLAY AL': 1027,
 'CLEBURNE AL': 1029,
 'COFFEE AL': 1031,
 'COLBERT AL': 1033,
 'CONECUH AL': 1035,
 'COOSA AL': 1037,
 'COVINGTON AL': 1039,
 'CRENSHAW AL': 1041,
 'CULLMAN AL': 1043,
 'DALE AL': 1045,
 'DALLAS AL': 1047,
 'DE KALB AL': 1049,
 'ELMORE AL': 1051,
 'ESCAMBIA AL': 1053,
 'ETOWAH AL': 1055,
 'FAYETTE AL': 1057,
 'FRANKLIN AL': 1059,
 'GENEVA AL': 1061,
 'GREENE AL': 1063,
 'HALE AL': 1065,
 'HENRY AL': 1067,
 'HOUSTON AL': 1069,
 'JACKSON AL': 1071,
 'JEFFERSON AL': 1073,
 'LAMAR AL': 1075,
 'LAUDERDALE AL': 1077,
 'LAWRENCE AL': 1079,
 'LEE AL': 1081,
 'LIMESTONE AL': 1083,
 'LOWNDES AL': 1085,
 'MACON AL': 1087,
 'MADISON AL': 1089,
 'MARENGO AL': 1091,
 'MARION AL': 1093,
 'MARSHALL AL': 1095,
 'MOBILE 

In [0]:
df['County'] == 

In [209]:
df['FIPS County Code']

0        2013.000
1        2013.000
2        2013.000
3        2013.000
4        2013.000
5        2013.000
6        2013.000
7        2013.000
8        2013.000
9        2013.000
10       2013.000
11       2013.000
12       2013.000
13       2013.000
14       2013.000
15       2013.000
16       2013.000
17       2013.000
18       2013.000
19       2013.000
20       2013.000
21       2013.000
22       2013.000
23       2013.000
24       2013.000
25       2013.000
26       2013.000
27       2013.000
28       2013.000
29       2013.000
           ...   
75307   56043.000
75308   56043.000
75309   56043.000
75310   56043.000
75311   56043.000
75312   56043.000
75313   56043.000
75314   56043.000
75315   56043.000
75316   56043.000
75317   56043.000
75318   56043.000
75319   56045.000
75320   56045.000
75321   56045.000
75322   56045.000
75323   56045.000
75324   56045.000
75325   56045.000
75326   56045.000
75327   56045.000
75328   56045.000
75329   56045.000
75330   56045.000
75331   56

In [0]:
df.groupby(['Plan Type'], as_index=True)['Avg Premium'].mean()

Plan Type
EPO   372.158
HMO   305.631
POS   360.832
PPO   372.962
Name: Avg Premium, dtype: float64

# Create Dataframes for Analysis and Visualization

In [0]:
# function to generate new dataframes for stats
def feature_by_premium(feature):

    new_df = pd.DataFrame()
       
    avg_prem_state = df.groupby(feature, as_index=True)['Avg Premium'].mean()
    new_df['avg_prem'] = avg_prem_state
    
    new_df['id'] = new_df.index    
  
    return new_df

In [0]:
state_prem_df = feature_by_premium('FIPS State Code')
state_prem_df = state_prem_df.reindex(list(range(state_prem_df.index.min(),state_prem_df.index.max()+1)),fill_value=np.nan)
state_prem_df['id'] = state_prem_df.index.astype('uint16')
state_prem_df['state'] = state_prem_df['id'].map(state_geo_dict)

In [46]:
state_prem_df.head()

,avg_prem,id,state
FIPS State Code,,,
1,209.674,1,AL
2,331.407,2,AK
3,nan,3,NaN
4,255.892,4,AZ
5,245.556,5,AR


In [45]:
county_prem_df = feature_by_premium('FIPS County Code')
county_prem_df.head()

,avg_prem,id
FIPS County Code,,
1001,212.456,1001
1003,205.675,1003
1005,205.675,1005
1007,226.016,1007
1009,226.016,1009


In [49]:
state_prem_df

,avg_prem,id,state
FIPS State Code,,,
1,209.674,1,AL
2,331.407,2,AK
3,nan,3,NaN
4,255.892,4,AZ
5,245.556,5,AR
6,nan,6,CA
7,nan,7,NaN
8,nan,8,CO
9,nan,9,CT


# Visualizations - Maps

In [52]:
states = alt.topo_feature(data.us_10m.url, 'states')
source = state_prem_df

foreground = alt.Chart(states).mark_geoshape(stroke='black').encode(
    color=alt.Color('avg_prem:Q', sort="descending",  scale=alt.Scale(scheme='viridis'), legend=alt.Legend(title="Avr Premium", tickCount=6))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'id', ['avg_prem'])
).project(
    type='albersUsa'
).properties(
    title="Average Monthly Individual Medical Insurance Premium by State", 
    width=700,
    height=350
)  


background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='black'
).properties(
    width=700,
    height=350
).project('albersUsa')

config = alt.layer(background, foreground).configure_title(fontSize=20, anchor="middle").configure_legend(titleColor='black', titleFontSize=14) 

config

LayerChart({
  config: Config({
    legend: LegendConfig({
      titleColor: 'black',
      titleFontSize: 14
    }),
    title: VgTitleConfig({
      anchor: 'middle',
      fontSize: 20
    })
  }),
  layer: [Chart({
    data: UrlData({
      format: TopoDataFormat({
        feature: 'states',
        type: 'topojson'
      }),
      url: 'https://vega.github.io/vega-datasets/data/us-10m.json'
    }),
    height: 350,
    mark: MarkDef({
      fill: 'lightgray',
      stroke: 'black',
      type: 'geoshape'
    }),
    projection: Projection({
      type: 'albersUsa'
    }),
    width: 700
  }), Chart({
    data: UrlData({
      format: TopoDataFormat({
        feature: 'states',
        type: 'topojson'
      }),
      url: 'https://vega.github.io/vega-datasets/data/us-10m.json'
    }),
    encoding: EncodingWithFacet({
      color: Color({
        legend: Legend({
          tickCount: 6,
          title: 'Avr Premium'
        }),
        scale: Scale({
          scheme: 'viridis'
        }),
        shorthand: 'avg_prem:Q',
        sort: 'descending'
      })
    }),
    height: 350,
    mark: MarkDef({
      stroke: 'black',
      type: 'geoshape'
    }),
    projection: Projection({
      type: 'albersUsa'
    }),
    title: 'Average Monthly Individual Medical Insurance Premium by State',
    transform: [LookupTransform({
      from: LookupData({
        data:                  avg_prem  id state
        FIPS State Code                    
        1                 209.674   1    AL
        2                 331.407   2    AK
        3                     nan   3   NaN
        4                 255.892   4    AZ
        5                 245.556   5    AR
        6                     nan   6    CA
        7                     nan   7   NaN
        8                     nan   8    CO
        9                     nan   9    CT
        10                250.892  10    DE
        11                    nan  11    DC
        12                256.565  12    FL
        13                261.139  13    GA
        14                    nan  14   NaN
        15                    nan  15    HA
        16                    nan  16    ID
        17                242.271  17    IL
        18                257.994  18    IN
        19                223.345  19    IA
        20                194.154  20    KS
        21                    nan  21    KY
        22                258.439  22    LA
        23                274.171  23    ME
        24                    nan  24    MD
        25                    nan  25    MA
        26                236.352  26    MI
        27                    nan  27    MN
        28                291.772  28    MS
        29                236.780  29    MO
        30                207.428  30    MT
        31                206.977  31    NE
        32                    nan  32    NV
        33                215.103  33    NH
        34                296.892  34    NJ
        35                    nan  35    NM
        36                    nan  36    NY
        37                249.713  37    NC
        38                224.995  38    ND
        39                232.070  39    OH
        40                207.613  40    OK
        41                    nan  41    OR
        42                213.748  42    PA
        43                    nan  43   NaN
        44                    nan  44    RI
        45                214.493  45    SC
        46                299.444  46    SD
        47                208.304  47    TN
        48                223.097  48    TX
        49                201.599  49    UT
        50                    nan  50    VT
        51                565.566  51    VA
        52                    nan  52   NaN
        53                    nan  53    WA
        54                222.636  54    WV
        55                266.670  55    WI
        56                354.086  56    WY,
        fields: ['avg_prem'],
        key: 'id'
      }),
      lo

In [53]:
counties = alt.topo_feature(data.us_10m.url, 'counties')
county_source = county_prem_df

foreground = alt.Chart(counties).mark_geoshape(
    ).encode(
    color=alt.Color('avg_prem:Q', sort="descending",  scale=alt.Scale(scheme='plasma'), legend=alt.Legend(title="Avr Premium", tickCount=6))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(county_source, 'id', ['avg_prem'])
).project(
    type='albersUsa'
).properties(
    title="Average Monthly Individual Medical Insurance Premium by County",
    width=500,
    height=300
)

background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='black'
).properties(
    width=700,
    height=350
).project('albersUsa')

config = alt.layer(background, foreground).configure_title(fontSize=20, anchor="middle").configure_legend(titleColor='black', titleFontSize=14) 

config

LayerChart({
  config: Config({
    legend: LegendConfig({
      titleColor: 'black',
      titleFontSize: 14
    }),
    title: VgTitleConfig({
      anchor: 'middle',
      fontSize: 20
    })
  }),
  layer: [Chart({
    data: UrlData({
      format: TopoDataFormat({
        feature: 'states',
        type: 'topojson'
      }),
      url: 'https://vega.github.io/vega-datasets/data/us-10m.json'
    }),
    height: 350,
    mark: MarkDef({
      fill: 'lightgray',
      stroke: 'black',
      type: 'geoshape'
    }),
    projection: Projection({
      type: 'albersUsa'
    }),
    width: 700
  }), Chart({
    data: UrlData({
      format: TopoDataFormat({
        feature: 'counties',
        type: 'topojson'
      }),
      url: 'https://vega.github.io/vega-datasets/data/us-10m.json'
    }),
    encoding: EncodingWithFacet({
      color: Color({
        legend: Legend({
          tickCount: 6,
          title: 'Avr Premium'
        }),
        scale: Scale({
          scheme: 'plasma'
        }),
        shorthand: 'avg_prem:Q',
        sort: 'descending'
      })
    }),
    height: 300,
    mark: 'geoshape',
    projection: Projection({
      type: 'albersUsa'
    }),
    title: 'Average Monthly Individual Medical Insurance Premium by County',
    transform: [LookupTransform({
      from: LookupData({
        data:                   avg_prem     id
        FIPS County Code                 
        1001               212.456   1001
        1003               205.675   1003
        1005               205.675   1005
        1007               226.016   1007
        1009               226.016   1009
        1011               205.675   1011
        1013               205.675   1013
        1015               212.456   1015
        1017               205.675   1017
        1019               205.675   1019
        1021               226.016   1021
        1023               205.675   1023
        1025               205.675   1025
        1027               205.675   1027
        1029               205.675   1029
        1031               205.675   1031
        1033               196.634   1033
        1035               205.675   1035
        1037               205.675   1037
        1039               205.675   1039
        1041               205.675   1041
        1043               205.675   1043
        1045               205.675   1045
        1047               205.675   1047
        1049               205.675   1049
        1051               212.456   1051
        1053               205.675   1053
        1055               212.456   1055
        1057               205.675   1057
        1059               205.675   1059
        ...                    ...    ...
        55129              250.304  55129
        55131              285.255  55131
        55133              278.818  55133
        55135              248.629  55135
        55137              228.500  55137
        55139              243.232  55139
        55141              259.859  55141
        56001              355.720  56001
        56003              355.720  56003
        56005              355.720  56005
        56007              355.720  56007
        56009              355.720  56009
        56011              355.720  56011
        56013              355.720  56013
        56015              355.720  56015
        56017              355.720  56017
        56019              355.720  56019
        56021              336.929  56021
        56023              355.720  56023
        56025              336.929  56025
        56027              355.720  56027
        56029              355.720  56029
        56031              355.720  56031
        56033              355.720  56033
        56035              355.720  56035
        56037              355.720  56037
        56039              355.720  56039
        56041              355.720  56041
        56043              355.720  56043
        56045              355.720  56045
        
        [2510 

# Model

In [0]:
# shuffle data for cross_val_score; function scores poor w/o it
shuffle_df = df.reindex(np.random.permutation(df.index))

# created modified dataframe of only numeric values
mod_df = shuffle_df._get_numeric_data()

# eliminate features with near 100% correlation with y-var
mod_df = mod_df.drop(['Premium Child',
       'Premium Adult Individual Age 21', 'Premium Adult Individual Age 27',
       'Premium Adult Individual Age 30', 'Premium Adult Individual Age 40',
       'Premium Adult Individual Age 50', 'Premium Adult Individual Age 60',
       'Premium Couple 21', 'Premium Couple 30', 'Premium Couple 40',
       'Premium Couple 50', 'Premium Couple 60', 'Couple+1 child, Age 21',
       'Couple+1 child, Age 30', 'Couple+1 child, Age 40',
       'Couple+1 child, Age 50', 'Couple+2 children, Age 21',
       'Couple+2 children, Age 30', 'Couple+2 children, Age 40',
       'Couple+2 children, Age 50', 'Couple+3 or more Children, Age 21',
       'Couple+3 or more Children, Age 30',
       'Couple+3 or more Children, Age 40',
       'Couple+3 or more Children, Age 50', 'Individual+1 child, Age 21',
       'Individual+1 child, Age 30', 'Individual+1 child, Age 40',
       'Individual+1 child, Age 50', 'Individual+2 children, Age 21',
       'Individual+2 children, Age 30', 'Individual+2 children, Age 40',
       'Individual+2 children, Age 50',
       'Individual+3 or more children, Age 21',
       'Individual+3 or more children, Age 30',
       'Individual+3 or more children, Age 40',
       'Individual+3 or more children, Age 50'], axis=1)

# drop mean price measures
mod_df = mod_df.drop(['Issuer_MP',
       'State_MP', 'County_MP', 'RatingArea_MP', 'PlansPer100K_MP',
       'MarketingID_MP', 'PopDensity_MP', 'PopDensityBin_MP'], axis=1)

In [0]:
X = mod_df.drop(['Avg Premium'], axis=1)
y = mod_df['Avg Premium']

In [169]:
X.columns

Index(['Rating Area', 'Child Dental',
       'Medical Deductible - individual - standard',
       'Drug Deductible - individual - standard',
       'Medical Deductible -family - standard',
       'Drug Deductible - family - standard',
       'Medical Maximum Out Of Pocket - individual - standard',
       'Drug Maximum Out of Pocket - individual - standard',
       'Medical Maximum Out of Pocket - family - standard',
       'Drug Maximum Out of Pocket - Family  - standard',
       'Medical Deductible - individual - 73 percent',
       'Drug Deductible - individual - 73 percent',
       'Medical Deductible - family - 73 percent',
       'Drug Deductible - family - 73 percent', 'FIPS State Code',
       'Pop Density', 'Population', 'Area', 'StateFreq', 'StateID', 'CountyID',
       'IssuerID', 'PlanMarketingID', 'Network URL ID', 'Source Encoded',
       'Metal Level Encoded', 'Plan Type Encoded', 'PlansPer100K'],
      dtype='object')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=99)

In [0]:
# Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y_pred, y_test) : 
    if len(y_test) == len(y_pred):
        return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))
      
# Root mean squared error (RMSE)
def rmse(y_pred, y_test):
  return np.sqrt(mean_squared_error(y_test, y_pred))

In [172]:
# Linear Regression
lm = LinearRegression()
model_lm = lm.fit(X_train,y_train)
pred_lm = lm.predict(X_test)
result_lm = model_lm.score(X_test, y_test)
cv_scores_lm = cross_val_score(lm, X, y, cv=5, scoring='neg_mean_squared_log_error')
lm_rmsle = np.sqrt(np.abs(cv_scores_lm.mean()))

print("=== Linear Regression ===")
print("R2: " + str(result_lm))
print("RMSLE: " + str(rmsle(pred_lm, y_test)))
print("RMSE: " + str(rmse(pred_lm, y_test)))
print("Mean RMSLE Score: ", lm_rmsle.mean())
print("\n")

# # Random Forest Regression
rfr = RandomForestRegressor()
model_rfr = rfr.fit(X_train,y_train)
pred_rfr = rfr.predict(X_test)
result_rfr = model_rfr.score(X_test, y_test)
cv_scores_rfr = cross_val_score(rfr, X, y, cv=5, scoring='neg_mean_squared_log_error')
rfr_rmsle = np.sqrt(np.abs(cv_scores_rfr.mean()))


print("=== Random Forest Regression ===")
print("R2: " + str(result_rfr))
print("RMSLE: " + str(rmsle(pred_rfr, y_test)))
print("RMSE: " + str(rmse(pred_rfr, y_test)))
print("Mean RMSLE Score: ", rfr_rmsle.mean())
print("\n")

# Extra Trees Regression
etr = ExtraTreesRegressor()
model_etr = etr.fit(X_train,y_train)
pred_etr = etr.predict(X_test)
result_etr = model_etr.score(X_test, y_test)
cv_scores_etr = cross_val_score(etr, X, y, cv=5, scoring='neg_mean_squared_log_error')
etr_rmsle = np.sqrt(np.abs(cv_scores_etr.mean()))

print("=== Extra Trees Regression ===")
print("R2: " + str(result_etr))
print("RMSLE: " + str(rmsle(pred_etr, y_test)))
print("RMSE: " + str(rmse(pred_etr, y_test)))
print("Mean RMSLE Score: ", etr_rmsle.mean())
print("\n")

=== Linear Regression ===
R2: 0.16896280916443074
RMSLE: 0.3490530527706113
RMSE: 158.4531999678413
Mean RMSLE Score:  0.3479769567944862


=== Random Forest Regression ===
R2: 0.9886695948089664
RMSLE: 0.04402934326500927
RMSE: 18.5017796698064
Mean RMSLE Score:  0.04208903126216028


=== Extra Trees Regression ===
R2: 0.9859972945107005
RMSLE: 0.05057567732809813
RMSE: 20.568222493851543
Mean RMSLE Score:  0.041072932301725325




In [173]:
# feature importance of random forest model
rf_fi=RandomForestRegressor()
rf_fi.fit(X, y)
FI_rf = pd.DataFrame({"Feature Importance":rf_fi.feature_importances_}, index=X.columns)
FI_rf_sorted = FI_rf.sort_values("Feature Importance",ascending=False)
print(FI_rf_sorted.to_string())

                                                    Feature Importance
PlanMarketingID                                                  0.664
FIPS State Code                                                  0.119
Metal Level Encoded                                              0.058
Network URL ID                                                   0.018
Medical Deductible -family - standard                            0.018
Plan Type Encoded                                                0.016
Medical Deductible - family - 73 percent                         0.012
IssuerID                                                         0.012
Rating Area                                                      0.012
CountyID                                                         0.008
Medical Deductible - individual - 73 percent                     0.008
Drug Deductible - individual - standard                          0.007
Drug Deductible - family - standard                              0.007
Drug D